In [1]:
import pandas as pd
import numpy as np

# 1 Average travel time between each intersection
***

## 1.1 Average number of frames in each section (with and without waiting for traffic light)

In [114]:
df = pd.read_csv('NGSIM-Data/trajectories-0400pm-0415pm_editted.csv')
df = df[(df['Tot_Frames'] > 20) & (df['Intersection'] == 0) & (df['Direction'] == 2)].fillna(0)
df_groupby_section = df.groupby(['Section', 'Vehicle_ID'])['Frame_ID', 'Veh_Velocity']
def get_time(df):
    idx = (df['Veh_Velocity'].values <=0).argmax()
    if idx == 0: idx = len(df)
    frames_withLight = df.iloc[0:idx]['Frame_ID'].max() - df.iloc[0:idx]['Frame_ID'].min()
    frames_withoutLight = df['Frame_ID'].max() - df['Frame_ID'].min()
    return pd.DataFrame(columns=['Frames_withLight', 'Frames_withoutLight'], data = np.vstack([frames_withLight, frames_withoutLight]).T)
section_meanTime = df_groupby_section.apply(get_time)#.groupby(level='Section').mean()
section_meanTime = section_meanTime.groupby(level='Section').mean()
section_meanTime

,Frames_withLight,Frames_withoutLight
Section,,
0,1.950000,1.950000
1,55.744361,141.360902
2,189.120567,234.560284
3,211.804348,297.260870
4,99.899329,100.973154
5,231.935065,392.753247
6,0.600000,0.600000


## 1.2 Average number of frames in each intersection
***

In [110]:
df = pd.read_csv('NGSIM-Data/trajectories-0400pm-0415pm_editted.csv')
df = df[(df['Tot_Frames'] > 20) & (df['Intersection'] != 0) & (df['Direction'] == 2)].fillna(0)
df_groupby_intersection = df.groupby(['Intersection', 'Vehicle_ID'])['Frame_ID']
intersection_meanTime = pd.DataFrame(df_groupby_intersection.agg(lambda x: x.max() - x.min()).groupby(level='Intersection').mean())
intersection_meanTime.rename(columns={'Frame_ID':'Mean Frames'}, inplace=True)
intersection_meanTime

,Mean Frames
Intersection,
1,26.595628
2,63.466667
3,45.513699
4,18.176871
5,60.763285


## 1.3 Adding up average time in intersections and sections 
***

In [112]:
time_array = np.array([section_meanTime.loc[1, 'Frames_withoutLight'], 
                       section_meanTime.loc[2, 'Frames_withoutLight'] + intersection_meanTime.loc[1], 
                       section_meanTime.loc[3, 'Frames_withoutLight'] + intersection_meanTime.loc[2], 
                       section_meanTime.loc[4, 'Frames_withLight'] + intersection_meanTime.loc[3] + 
                       section_meanTime.loc[5, 'Frames_withoutLight'] + intersection_meanTime.loc[4], 
                       section_meanTime.loc[6, 'Frames_withoutLight'] + intersection_meanTime.loc[5]])
indices = ['<1', '1-2', '2-3', '3-5', '>5']
time_df = pd.DataFrame(data=time_array/10, index = indices, columns = ['Time (seconds)'])
time_df.index.set_names(names=['Intersection'], inplace=True)
time_df

,Time (seconds)
Intersection,
<1,14.136090
1-2,26.115591
2-3,36.072754
3-5,55.634314
>5,6.136329


# 2 Flow distribution from each direction per intersection

Read file, filter cases with `Tot_Frames` > 20 and `Direction` == 2 (i.e, *Northbound* only), and remove vehicles traveling in **Sections**. 

In [ ]:
df = pd.read_csv('NGSIM-Data/trajectories-0400pm-0415pm_editted.csv')
df = df[(df['Tot_Frames'] > 20) & (df['Intersection'] != 0) & (df['Direction'] == 2)].fillna(0)

In [ ]:
sub_df = df[['Vehicle_ID', 'Frame_ID', 'Direction', 'Intersection', 'Movement']]
grouped_series = sub_df.sort_values('Frame_ID').groupby(['Intersection', 'Movement', 'Vehicle_ID'])['Frame_ID']

minFrame_df = pd.DataFrame(grouped_series.min()).reset_index()  # use min to only get first frame of each car in each group
g = minFrame_df[['Intersection', 'Movement', 'Frame_ID']].groupby(['Intersection', 'Movement']) # don't have to group by VehicleID anymore because only one frame per vehicle
dFrame_series = g.apply(lambda x: x.sort_values(by='Frame_ID',ascending=True).diff())

dFrame_df = pd.DataFrame(dFrame_series['Frame_ID']).reset_index().rename(columns={'Frame_ID':'DFrame'}).drop(columns='level_2')
grouped_results = dFrame_df.groupby(['Intersection', 'Movement'])
frames_per_bin = 50
bin_edges = np.arange(0,501,frames_per_bin)

Calculate probability density and write to Distributions.csv
***

In [ ]:
with open("Distributions.csv", mode='ab') as dist_file:
    for i, (group_name, gdf) in enumerate(grouped_results): 
        hist, edges = np.histogram(gdf['DFrame'], bins=bin_edges, density=True)
        np.savetxt(dist_file, X=np.vstack([bin_edges[1:]/10, hist*frames_per_bin]).T, fmt='%.6f', delimiter=',', header=f"{group_name[0]},{group_name[1]},{len(hist)}")

# 5 Travel time from different intersection (or the first intersection) to the end


# 6 Average entering speed for CA 

In [118]:
df = pd.read_csv('NGSIM-Data/trajectories-0400pm-0415pm_editted.csv')
df = df[(df['Tot_Frames'] > 20) & (df['Section'] == 1) & (df['Direction'] == 2)].fillna(0)
grouped_df = df.groupby(['Vehicle_ID'])
agg_df = grouped_df.apply(lambda x: x.iloc[x['Frame_ID'].values.argmin(),:])
print(f"Average entering velocity (ft/s) = {agg_df['Veh_Velocity'].mean():.3f}")

Average entering velocity (ft/s) = 22.494


# 7 Probabilities of departure at every intersection 

In [ ]:
df = pd.read_csv('NGSIM-Data/trajectories-0400pm-0415pm_editted.csv')
df = df[(df['Tot_Frames'] > 20) & (df['Intersection'] != 0) & (df['Direction'] == 2)].fillna(0)
def get_outputs(x):
    cols = ['Frame_ID', 'Intersection', 'Movement']
    x['Frame_ID'] = x['Frame_ID'].values/10
    start_df = x.loc[x['Frame_ID'].idxmin(), cols].rename(dict(zip(cols,['Start_Time (s)', 'Start_Intersection', 'Start_Direction'])))
    end_df = x.loc[x['Frame_ID'].idxmax(), cols].rename(dict(zip(cols,['End_Time (s)', 'End_Intersection', 'End_Direction'])))
    return pd.concat([start_df, end_df],axis=0)
vehicle_group = df[['Vehicle_ID', 'Frame_ID', 'Intersection', 'Movement']].groupby('Vehicle_ID')
vehicle_group.apply(get_outputs)
outfile = vehicle_group.apply(get_outputs)
outfile.to_csv("output_trafficsim.csv")